# **`Import`**

In [ ]:
!pip install torchmetrics

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torchmetrics import Accuracy, HingeLoss

# **`Device`**

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# **`Laod Dataset`**

In [4]:
data_frame_train = pd.read_csv('/content/train.csv')
date_frame_test = pd.read_csv('/content/test.csv')

# **`Train`**

In [5]:
x_train = data_frame_train.drop('price_range', axis=1).values
y_train = data_frame_train['price_range'].values

# **`Split`**

In [6]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, train_size=0.7, random_state=42)

# **`Preprocess`**

In [7]:
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)


# for validation

x_valid = torch.FloatTensor(x_valid)
y_valid = torch.LongTensor(y_valid)


# **`Standardization`**

In [8]:
mu = x_train.mean(dim=0)
std = x_train.std(dim=0)

In [9]:
x_train = (x_train - mu) / std
x_valid = (x_valid - mu) / std

# **`Dataloader`**

In [10]:
train_data = TensorDataset(x_train, y_train)
valid_data = TensorDataset(x_valid, y_valid)

In [11]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=64)

# **`Utils`**

In [18]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# **`Model`**

In [34]:
num_feats = 20
num_class = 4
hiden_layer_one = 64
hiden_layer_two = 32

model = nn.Sequential(nn.Linear(num_feats, hiden_layer_one),
                      nn.ReLU(),
                      nn.Linear(hiden_layer_one, hiden_layer_two),
                      nn.ReLU(),
                      nn.Linear(hiden_layer_two, num_class)
                      )

model = model.to(device)


# **`Loss and Optimizer`**


In [35]:
loss_fn = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True)
# optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.99)
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))



# **`Train loop`**

In [ ]:
from os import listxattr
num_epochs = 400

loss_train_hist = []
loss_valid_hist = []

acc_train_hist = []
acc_valid_hist = []


for epoch in range(num_epochs):
    loss_train = AverageMeter()
    acc_train = Accuracy(task='multiclass', num_classes=4).to(device)
    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)

        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss_train.update(loss.item())
        acc_train(outputs, targets)




    with torch.no_grad():
        loss_valid = AverageMeter()
        acc_valid = Accuracy(task='multiclass', num_classes=4).to(device)

        for i, (inputs, targets) in enumerate(valid_loader):
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            # valid_loss.append(loss)
            loss_valid.update(loss.item())

            acc_valid(outputs, targets)

    loss_train_hist.append(loss_train.avg)
    loss_valid_hist.append(loss_valid.avg)

    acc_train_hist.append(acc_train.compute())
    acc_valid_hist.append(acc_valid.compute())




    if epoch % 10 == 0:

        print(f'Epoch {epoch}')

        print(f'Train: Loss = {loss_train.avg:.4}, Acc = {acc_train.compute():.4}')

        print(f'Valid: Loss = {loss_valid.avg:.4}, Acc = {acc_valid.compute():.4}', "\n")



# **`Plot`**

# **`Loss`**

In [ ]:
plt.plot(range(num_epochs), loss_train_hist, 'r-', label='Train')
plt.plot(range(num_epochs), loss_valid_hist, 'b-', label='Validation')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

# **`Accuracy`**

In [ ]:
plt.figure(figsize=(8, 6))

plt.plot(range(num_epochs), acc_train_hist, 'r-', label='Train')
plt.plot(range(num_epochs), acc_valid_hist, 'b-', label='Validation')

plt.xlabel('Epoch')
plt.ylabel('Acc')
plt.grid(True)
plt.legend()